# Bronchoscopy dataset

### available features
* demo - age, sex, BMI
* risk factors - fhist, phist of any cancer or past 5 years, COPD, emphysema, pkyr, smoking quit time, smoking status, nodule spiculation, nodule location
* nodule pathology

missing: race, education

In [1]:
import pandas as pd
import os
from datetime import datetime
import glob

In [29]:
cli = "/home/local/VANDERBILT/litz/data/bronch/bronch_raw.xlsm"
dates = "/home/local/VANDERBILT/litz/data/bronch/bronch_dates.xlsx"
cli = pd.read_excel(cli, dtype={'MRN':str}) 
dates = pd.read_excel(dates, dtype={'MRN':str, 'MCL ID':str})
print(f"subjects in clinical data: {len(cli['MRN'].unique())}")
print(f"subjects in dates data: {len(dates['MRN'].unique())}")

# subjects may have multiple nodules - group by largest nodule
largest_nodule = cli.iloc[cli.groupby('MRN')['Size of Nodule (Specific) (cm)'].idxmax()]

sessions = largest_nodule.merge(dates, on='MRN')
sessions

subjects in clinical data: 387
subjects in dates data: 391


,Record ID_x,MRN,Final nodule diagnosis,Age,Sex,Size of Nodule (Specific) (cm),Smoking Status,Does the patient have a history of extrathoracic cancer?,History of cancer within the past five years?,Location of nodule/mass,...,How was the nodule detected? Ensure you check the original reference scan in PACS,Primary purpose of reference bronchoscopy,Record ID_y,MCL ID,Date of Procedure,Size of Nodule (cm),HM size,Location,Density_y,Ref CT date
0,407,10053866,Benign,62,female,0.8,Current,No,No,RUL,...,screen-detected,diagnostic,407,27054566930,2018-10-12 00:00:00,0.8,8.5,RUL,Solid,2018-10-04
1,209,10133197,Malignant,61,male,2.5,Former,No,No,LUL,...,detected during active surveillance,tissue sampling for molecular analysis (alread...,209,20802811825,2018-07-27 00:00:00,2.5,2.6,LUL,Solid,2018-06-12
2,209,10133197,Malignant,61,male,2.5,Former,No,No,LUL,...,detected during active surveillance,tissue sampling for molecular analysis (alread...,220001,20802811825,2018-06-29 00:00:00,2.3,2.5,LUL,Solid,2019-06-10
3,209,10133197,Malignant,61,male,2.5,Former,No,No,LUL,...,detected during active surveillance,tissue sampling for molecular analysis (alread...,220002,20802811825,2018-06-29 00:00:00,1.2,1.2,RUL,Solid,2019-06-10
4,376,10227932,Benign,57,female,2.4,Former,No,No,LLL,...,incidentally detected,diagnostic,376,24540308812,2018-11-01 00:00:00,2.4,2.6,LLL,Solid,2018-10-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,285,9317355,Benign,79,male,2.1,Former,No,No,LLL,...,incidentally detected,diagnostic,285,34087001117,2019-03-15 00:00:00,2.1,2.3,LLL,Solid,2019-02-20
444,107,9608969,Malignant,59,male,1.5,Former,No,No,LUL,...,incidentally detected,diagnostic,107,5434231673,2018-05-16 00:00:00,1.5,1.4,LUL,Solid,2018-04-17
445,287,9683517,Benign,56,male,3.6,Former,No,No,LLL,...,incidentally detected,diagnostic,287,450204236,2019-03-13 00:00:00,3.6,3.9,LLL,Solid,2019-03-01
446,69,9822032,Benign,75,female,1.9,Former,No,No,RML,...,incidentally detected,diagnostic,69,41666093991,2018-03-01 00:00:00,1.9,1.9,RML,Solid,2018-02-21


In [30]:
xnat_path = '/home/local/VANDERBILT/litz/data/ajrccm/xnat20221201/MCL_CT.csv'
xnat = pd.read_csv(xnat_path)
def impute_date(x):
    if pd.isnull(x['Date']):
        try:
            return pd.to_datetime(x['XNAT_CTSESSIONDATA ID'].split('_')[1], format='%Y-%m-%d')
        except:
            return None
    else:
        return pd.to_datetime(x['Date'], format='%Y-%m-%d')
xnat['Date'] = xnat.apply(lambda x: impute_date(x), axis=1)
xnat['Date'] = pd.to_datetime(xnat['Date'], format='%Y-%m-%d')
sessions['Ref CT date'] = pd.to_datetime(sessions['Ref CT date'], format='%Y-%m-%d')
sessions[['MCL ID', 'Date', 'Ref CT date']]

KeyError: "['Date'] not in index"

In [4]:
# bronch_xnat = xnat.merge(sessions, left_on=['Subject', 'Date'], right_on=['MCL ID', 'Ref CT date'])
# print(f"num sessions: {len(bronch_xnat)}")
# print(f"num subjects: {len(bronch_xnat['MCL ID'].unique())}")
# sess_str =','.join(bronch_xnat['XNAT_CTSESSIONDATA ID'].tolist())
# download = "/home/local/VANDERBILT/litz/data/bronch/bronch_xnat_sessions.txt"
# with open(download, 'w') as f:
#     f.write(sess_str)


### Cases and controls
1. match scan with nodule record via Record ID
2. group by subject 

In [5]:
print(len(dates))
print(len(dates['MCL ID'].unique()))
print(len(cli))

454
391
450


In [9]:
nodule_counts = cli.groupby('MRN', as_index=False)['Sex'].count().rename(columns={'Sex':'nodule_count'})
cli = cli.merge(nodule_counts, on='MRN')
cli_dates = cli.merge(dates, on=['MRN','Record ID'])

# cross sectional cohort
cli_dates['lung_cancer'] = cli_dates['Final nodule diagnosis'].apply(lambda x: int(x=='Malignant'))
cs = cli_dates.iloc[cli_dates.groupby(['MRN'])['lung_cancer'].idxmax()]
cs

,Record ID,MRN,Final nodule diagnosis,Age,Sex,Size of Nodule (Specific) (cm),Smoking Status,Does the patient have a history of extrathoracic cancer?,History of cancer within the past five years?,Location of nodule/mass,...,nodule_count_y,nodule_count,MCL ID,Date of Procedure,Size of Nodule (cm),HM size,Location,Density_y,Ref CT date,lung_cancer
345,407,10053866,Benign,62,female,0.8,Current,No,No,RUL,...,1,1,27054566930,2018-10-12 00:00:00,0.8,8.5,RUL,Solid,2018-10-04,0
173,209,10133197,Malignant,61,male,2.5,Former,No,No,LUL,...,3,3,20802811825,2018-07-27 00:00:00,2.5,2.6,LUL,Solid,2018-06-12,1
318,376,10227932,Benign,57,female,2.4,Former,No,No,LLL,...,1,1,24540308812,2018-11-01 00:00:00,2.4,2.6,LLL,Solid,2018-10-25,0
305,363,10244135,Malignant,71,female,3.2,Former,No,No,LLL,...,1,1,29479908093,2018-12-06 00:00:00,3.2,3.0,LLL,Solid,2018-12-03,1
311,369,10567634,Benign,74,female,3.4,Never,Yes,Yes,RUL,...,1,1,14583998995,2018-12-28 00:00:00,3.4,3.4,RUL,Solid,2018-12-07,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,285,9317355,Benign,79,male,2.1,Former,No,No,LLL,...,1,1,34087001117,2019-03-15 00:00:00,2.1,2.3,LLL,Solid,2019-02-20,0
87,107,9608969,Malignant,59,male,1.5,Former,No,No,LUL,...,1,1,5434231673,2018-05-16 00:00:00,1.5,1.4,LUL,Solid,2018-04-17,1
236,287,9683517,Benign,56,male,3.6,Former,No,No,LLL,...,1,1,450204236,2019-03-13 00:00:00,3.6,3.9,LLL,Solid,2019-03-01,0
54,69,9822032,Benign,75,female,1.9,Former,No,No,RML,...,1,1,41666093991,2018-03-01 00:00:00,1.9,1.9,RML,Solid,2018-02-21,0


### Radiologic features

In [15]:
def get_spiculation(x):
    map = {'Checked':1, 'Unchecked':0}
    return map[x]
def get_upper_lobe(x):
    map = {
        'RUL':1,
        'RML':0,
        'RLL':0,
        'LUL':1,
        'Lingula':0,
        'LLL':0
    }
    return map[x]
def get_nodule_type(x):
    map = {
        'Pure GGO': 0,
        'Part-solid': 1,
        'Solid': 2
    }
    return map[x]

cs['spiculation'] = cs['Nodule edge characteristics (choice=Spiculation)'].apply(lambda x: get_spiculation(x))
cs['upper_lobe'] = cs['Location'].apply(lambda x: get_upper_lobe(x))
cs['nodule_size'] = cs['Size of Nodule (Specific) (cm)'].astype(float)
cs['nodule_type'] = cs['Density_x'].apply(lambda x: get_nodule_type(x))
cs['nodule_count'] = cs['nodule_count']

/tmp/ipykernel_3401216/3501071108.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cs['spiculation'] = cs['Nodule edge characteristics (choice=Spiculation)'].apply(lambda x: get_spiculation(x))
/tmp/ipykernel_3401216/3501071108.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cs['upper_lobe'] = cs['Location'].apply(lambda x: get_upper_lobe(x))
/tmp/ipykernel_3401216/3501071108.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

### Clinical features

In [26]:
cs['bmi']

345    21.6
173    20.9
318    22.1
305    22.7
311    25.3
       ... 
234    25.8
87     24.4
236    38.3
54     18.0
247    34.2
Name: bmi, Length: 385, dtype: float64

In [23]:
def get_sex(x):
    map = {
        'male':1,
        'female':0
    }
    return map[x]
def parse_yesno(x):
    map = {
        'Yes':True,
        'No':False
    }
    return map[x]

def get_smo_status(x):
    map = {
        'Former': 0,
        'Current':1,
        'Never':2
    }
    return map[x]

cs['age'] = cs['Age']
cs['sex'] = cs['Sex'].apply(lambda x: get_sex(x))
cs['bmi'] = cs['BMI:'].astype(float)
cs['phist_extrathoracic'] = cs['Does the patient have a history of extrathoracic cancer?'].apply(lambda x: parse_yesno(x))
cs['phist_primary'] = cs['Does the patient have a history of prior primary lung cancer?'].apply(lambda x: parse_yesno(x))
cs['phist'] = (cs['phist_extrathoracic'] | cs['phist_primary']).astype(int)
cs['fhist'] = cs['Family history of lung cancer?'].apply(lambda x: parse_yesno(x)).astype(int)
cs['emphysema'] = cs['Radiographic emphysema present?'].apply(lambda x: parse_yesno(x)).astype(int)
cs['copd'] = cs['History of COPD?'].apply(lambda x: parse_yesno(x)).astype(int)
cs['smo_status'] = cs['Smoking Status'].apply(lambda x: get_smo_status(x))
cs['pkyr'] = cs['Pack year history of smoking:']
cs['quit_time'] = cs['Years since quitting smoking: Enter 0 if still active smoker at time of bronchoscopy']


/tmp/ipykernel_3401216/2045154213.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cs['age'] = cs['Age']
/tmp/ipykernel_3401216/2045154213.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cs['sex'] = cs['Sex'].apply(lambda x: get_sex(x))
/tmp/ipykernel_3401216/2045154213.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

In [137]:
clinical_ft = ['pid', 'age', 'sex', 'bmi', 'emphysema', 'copd', 'phist', 'fhist', 'smo_status', 'quit_time', 'pkyr', 'lung_cancer']
nodule_ft = ['spiculation', 'upper_lobe', 'nodule_size', 'nodule_type', 'nodule_count']
cs['pid'] = cs['MCL ID']
cs = cs[clinical_ft+nodule_ft]
cs

/tmp/ipykernel_3174113/3345588882.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cs['pid'] = cs['MCL ID']


,pid,age,sex,bmi,emphysema,copd,phist,fhist,smo_status,quit_time,pkyr,lung_cancer,spiculation,upper_lobe,nodule_size,nodule_type,nodule_count
345,27054566930,62,0,21.6,1,1,0,0,1,0.0,40.0,0,0,1,0.8,2,1
173,20802811825,61,1,20.9,1,1,1,0,0,3.0,66.0,1,0,1,2.5,2,3
318,24540308812,57,0,22.1,1,1,0,0,0,0.0,60.0,0,0,0,2.4,2,1
305,29479908093,71,0,22.7,1,1,0,1,0,10.0,40.0,1,1,0,3.2,2,1
311,14583998995,74,0,25.3,0,0,1,0,2,NaN,0.0,0,0,1,3.4,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,34087001117,79,1,25.8,0,0,0,1,0,40.0,20.0,0,0,0,2.1,2,1
87,5434231673,59,1,24.4,0,1,0,0,0,0.0,15.0,1,1,1,1.5,2,1
236,450204236,56,1,38.3,0,0,0,0,0,38.0,NaN,0,0,0,3.6,2,1
54,41666093991,75,0,18.0,0,0,0,0,0,35.0,NaN,0,0,0,1.9,2,1


### Get nifti scans

In [138]:
# scan_dir = "/nfs/masi/MCL/xnat/xnat20221207_bronch/bronch"
# rows = []
# for mcl in os.listdir(scan_dir):
#     for scanid in os.listdir(os.path.join(scan_dir, mcl)):
#         try:
#             scan_date = datetime.strptime(scanid, "%Y%m%d")
#             for fname in os.listdir(os.path.join(scan_dir, mcl, scanid)):
#                 rows.append({'pid': mcl, 'scan_date': scan_date, 'filename':fname})
#         except:
#             continue

prep_dir = "/home/local/VANDERBILT/litz/data/bronch/DeepLungScreening/prep/"
rows = []
for p in glob.glob(os.path.join(prep_dir, "*_clean.nii.gz")):
    scanid = os.path.basename(p).split('_clean.nii.gz')[0]
    mcl, date = scanid.split('time')
    scan_date = datetime.strptime(date, "%Y%m%d")
    rows.append({'pid': mcl, 'scan_date': scan_date, 'filename': scanid})


scan_df = pd.DataFrame(rows)
scan_df = scan_df.sort_values(by=['pid', 'scan_date'])
scan_df['session'] = scan_df.groupby(['pid'])['scan_date'].rank('dense', ascending=True) # assign a T0, T1, or T2 for each scan
scan_df['session'] = scan_df['session'] - 1
scan_df['session'] = scan_df['session'].astype(int)
scan_df

,pid,scan_date,filename,session
162,10140118338,2018-10-19,10140118338time20181019,0
190,10232218755,2019-02-04,10232218755time20190204,0
334,10302863632,2019-02-16,10302863632time20190216,0
155,10310069205,2018-10-05,10310069205time20181005,0
250,10501496583,2019-01-08,10501496583time20190108,0
...,...,...,...,...
218,8666755991,2017-12-01,8666755991time20171201,0
186,9057763661,2018-08-10,9057763661time20180810,0
136,9345567287,2018-07-13,9345567287time20180713,0
345,9473800086,2018-08-13,9473800086time20180813,0


In [139]:
cohort = cs.merge(scan_df, on='pid')
cohort['id'] = cohort['filename'].apply(lambda x: x.split('.nii.gz')[0])
cohort['with_marker'] = 1
cohort['with_image'] = 1
cohort_path = "/home/local/VANDERBILT/litz/github/MASILab/DeepLungScreening/cohorts/bronch/bronch_prep_v1.csv"
cohort.to_csv(cohort_path, index=False)

## Cross sectional cohort

In [140]:
xsectional = cohort.loc[cohort.groupby('pid')['scan_date'].idxmax()]
xsectional_path = "/home/local/VANDERBILT/litz/github/MASILab/DeepLungScreening/cohorts/bronch/bronch_cs_prep_v1.csv"
xsectional.to_csv(xsectional_path, index=False)

In [141]:
xsectional

,pid,age,sex,bmi,emphysema,copd,phist,fhist,smo_status,quit_time,...,upper_lobe,nodule_size,nodule_type,nodule_count,scan_date,filename,session,id,with_marker,with_image
40,10140118338,62,1,25.8,0,0,0,0,2,NaN,...,1,1.2,0,1,2018-10-19,10140118338time20181019,0,10140118338time20181019,1,1
359,10232218755,64,0,23.0,0,0,1,0,1,0.0,...,1,1.9,2,1,2019-02-04,10232218755time20190204,0,10232218755time20190204,1,1
94,10302863632,65,0,18.9,0,1,1,1,2,NaN,...,0,1.2,2,1,2019-02-16,10302863632time20190216,0,10302863632time20190216,1,1
324,10310069205,62,0,18.2,0,1,0,0,1,0.0,...,1,2.6,2,1,2018-10-05,10310069205time20181005,0,10310069205time20181005,1,1
8,10501496583,41,1,30.0,0,0,1,1,0,8.0,...,0,1.8,2,1,2019-01-08,10501496583time20190108,0,10501496583time20190108,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,8666755991,69,1,28.1,0,1,1,0,0,17.0,...,1,1.4,2,1,2017-12-01,8666755991time20171201,0,8666755991time20171201,1,1
207,9057763661,67,1,19.8,0,0,1,0,0,16.0,...,1,1.4,2,1,2018-08-10,9057763661time20180810,0,9057763661time20180810,1,1
7,9345567287,37,0,20.2,0,0,0,0,2,NaN,...,1,3.2,2,1,2018-07-13,9345567287time20180713,0,9345567287time20180713,1,1
311,9473800086,66,0,25.5,1,0,0,1,0,15.0,...,1,2.1,2,1,2018-08-13,9473800086time20180813,0,9473800086time20180813,1,1


In [6]:
# group nodules
bronch['lung_cancer'] = bronch['Final nodule diagnosis'].apply(lambda x: int(x=='Malignant'))
idxmax = bronch.groupby(['MRN'])['lung_cancer'].idxmax()
cohort = bronch.loc[idxmax]
cohort

,Record ID,MRN,Final nodule diagnosis,Age,Sex,Size of Nodule (Specific) (cm),Smoking Status,Does the patient have a history of extrathoracic cancer?,History of cancer within the past five years?,Location of nodule/mass,...,Pack year history of smoking:,History of COPD?,Peripheral 1/3 of lung,Growth of primary nodule noted on surveillance imaging? Choose No if no prior imaging,"Presence of pre-procedure symptoms? Includes: unexplained weight loss >5kg, dyspnea, pneumothorax, fatigue, pain or hemoptypsis",Pre-bronch percent predicted FEV1 (%): Enter as an integer 1-110,Years since quitting smoking: Enter 0 if still active smoker at time of bronchoscopy,How was the nodule detected? Ensure you check the original reference scan in PACS,Primary purpose of reference bronchoscopy,lung_cancer
343,407,10053866,Benign,62,female,0.8,Current,No,No,RUL,...,40.0,Yes,Yes,No,No,NaN,0.0,screen-detected,diagnostic,0
170,209,10133197,Malignant,61,male,2.5,Former,No,No,LUL,...,66.0,Yes,Yes,Yes,No,20.0,3.0,detected during active surveillance,tissue sampling for molecular analysis (alread...,1
315,376,10227932,Benign,57,female,2.4,Former,No,No,LLL,...,60.0,Yes,Yes,No,No,NaN,0.0,incidentally detected,diagnostic,0
302,363,10244135,Malignant,71,female,3.2,Former,No,No,LLL,...,40.0,Yes,Yes,No,Yes,56.0,10.0,incidentally detected,diagnostic,1
308,369,10567634,Benign,74,female,3.4,Never,Yes,Yes,RUL,...,0.0,No,Yes,Yes,No,NaN,NaN,incidentally detected,diagnostic,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,285,9317355,Benign,79,male,2.1,Former,No,No,LLL,...,20.0,No,Yes,Yes,No,NaN,40.0,incidentally detected,diagnostic,0
84,107,9608969,Malignant,59,male,1.5,Former,No,No,LUL,...,15.0,Yes,Yes,Yes,No,68.0,0.0,incidentally detected,diagnostic,1
234,287,9683517,Benign,56,male,3.6,Former,No,No,LLL,...,NaN,No,Yes,Yes,No,75.0,38.0,incidentally detected,diagnostic,0
52,69,9822032,Benign,75,female,1.9,Former,No,No,RML,...,NaN,No,Yes,Yes,No,NaN,35.0,incidentally detected,diagnostic,0


In [7]:
cohort['lung_cancer'].value_counts()

1    236
0    151
Name: lung_cancer, dtype: int64

In [8]:
bronch_path = "/home/local/VANDERBILT/litz/data/bronch/Bronch MCL.csv"
bronch = pd.read_csv(bronch_path, dtype={'MCL ID': str})
bronch = bronch.rename(columns={'MCL ID': 'mcl_id'})

In [17]:
bronch

NameError: name 'bronch' is not defined